<!-- dom:TITLE: Exercises from Linear algebra, signal processing, and wavelets. A unified approach.\\ Python version -->
# Exercises from Linear algebra, signal processing, and wavelets. A unified approach.\\ Python version
<!-- dom:AUTHOR: Øyvind Ryan -->
<!-- Author: --> **Øyvind Ryan**

Date: **Oct 24, 2018**

<!-- Externaldocuments: applinalg -->
<!-- Mapping from exercise labels to numbers: label2numbers = {'example:extractcolours': '8.2', 'exercise:dftdctonblocks': '8.24', 'example:chesspatternmoluecules': '8.12', 'exercise:contrastex': '8.7', 'exercise:genimgs': '8.13', 'example:rgb-grey': '8.3', 'example:edge_detection': '8.10', 'example:smoothing': '8.9', 'example:secondorder_derivative': '8.11', 'exercise:genbw': '8.6', 'example:negative': '8.4', 'example:contrast': '8.5', 'example:change_coords_dct2': '8.23', 'example:change_coords_dft2': '8.22', 'exercise:contrastimages3': '8.8'} -->

# Digital images














<!-- --- begin exercise --- -->

## Example 8.2: Extracting the different colors
<div id="example:extractcolours"></div>
<!-- keywords = imageschap -->

If we have a color image $P=(r_{i,j},g_{i,j},b_{i,j})_{i,j=1}^{m,n}$, it is often useful to manipulate the three color components separately as the three images

$$
P_r=(r_{i,j})_{i,j=1}^{m,n},\quad P_g=(g_{i,j})_{i,j=1}^{m,n},\quad P_b=(b_{i,j})_{i,j=1}^{m,n}.
$$

As an example, let us first see how we can produce three separate images, showing the red, green, and blue color components of the Lena image. First we need to read the file:

In [1]:
import os, sys
sys.path.append(os.path.join(os.getcwd(), 'python'))

In [2]:
%matplotlib inline

from images import *
from fft import *
from scipy.fftpack import dct, idct
from numpy import *
import matplotlib.pyplot as plt
from sound import filter_impl
from forward_compress_reverse import *

In [3]:
img = double(imread('images/lena.png', 'png'))

The returned object has three dimensions, the first two representing the spatial directions (row/column), the third the color component. 
By using different values for the third dimension we can thus obtain the different color components as follows:

In [4]:
X1 = zeros_like(img)
X1[:, :, 0] = img[:, :, 0]

X2 = zeros_like(img)
X2[:, :, 1] = img[:, :, 1]

X3 = zeros_like(img)
X3[:, :, 2] = img[:, :, 2]

In [5]:
imshow(uint8(X1))
imshow(uint8(X2))
imshow(uint8(X3))

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Example 8.3: Converting from color to grey-level
<div id="example:rgb-grey"></div>
<!-- keywords = imageschap -->

A color image can easily be converted to a grey-level image: At each point in the image we have to replace the three color values $(r,g,b)$ by a single value $q$ that will represent the grey level. 
If we want the grey-level image to be a reasonable representation of the color image, the value $q$ should somehow reflect the intensity of the image at the point. There are several ways to do this.
If the image is $P=(r_{i,j},g_{i,j},b_{i,j})_{i,j=1}^{m,n}$, we can (for all $i$ and $j$) either
* set $q_{i,j}=\max(r_{i,j},g_{i,j},b_{i,j})$, i.e. use the largest of the three color components,

* set $q_{i,j}=(r_{i,j}+g_{i,j}+b_{i,j})/3$, i.e. use the average of the three values,

* set $q_{i,j}=\sqrt{r_{i,j}^2+g_{i,j}^2+b_{i,j}^2}$, i.e. use the "length" of the color vector $(r,g,b)$.

The three methods can be implemented as follows.

In [6]:
mx = maximum(img[:, :, 0], img[:, :, 1])
X1 = maximum(img[:, :, 2], mx)

X2 = (img[:, :, 0] + img[ :, :, 1] + img[ :, :, 2])/3.

X3 = sqrt(img[:,:,0]**2 + img[:, :, 1]**2 + img[:, :, 2]**2)
map_to_01(X3)
X3 *= 255

For the third method we had to be careful, since $\sqrt{r^2+g^2+b^2}$ may be outside $[0,1]$, even if $r$, $g$ and $b$ all lie in $[0,1]$. We thus need to normalize the values as explained above.
In practice one of the last two methods are preferred, perhaps with a preference for the last method, but the actual choice depends on the application.

In [7]:
imshow(uint8(X1))
imshow(uint8(X2))
imshow(uint8(X3))

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Example 8.4: Computing the negative image
<div id="example:negative"></div>
<!-- keywords = imageschap -->

In film-based photography a negative image was obtained when the film was developed, and then a positive image was created from the negative. 
We can easily simulate this and compute a negative digital image.

Suppose we have a grey-level image $P=(p_{i,j})_{i,j=1}^{m,n}$ with intensity values in the interval $[0,1]$. Here intensity value 0 corresponds to black and 1 
corresponds to white. To obtain the negative image we just have to replace an intensity $p$ by its 'mirror value' $1-p$.
This is also easily translated to code as above.

In [8]:
X1 = 255 - X1
X2 = 255 - X2
X3 = 255 - X3

imshow(uint8(X1))
imshow(uint8(X2))
imshow(uint8(X3))

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Example 8.5: Adjusting the contrast
<div id="example:contrast"></div>
<!-- keywords = imageschap -->

A common problem with images is that the contrast often is not good enough. 
This typically means that a large proportion of the grey values are concentrated in a rather small subinterval of $[0,1]$. 
The obvious solution is to somehow spread out the values. 
This can be accomplished by applying a monotone function $f$ which maps $[0,1]$ onto $[0,1]$. 
If we choose $f$ so that its derivative is large in the area where many intensity values are concentrated, we obtain the desired effect. 
We will consider two such families of functions:

$$
\begin{align*}
f_n(x)        &= \frac{\arctan(n(x-1/2))}{2\arctan(n/2)}+ \frac{1}{2} \\ 
g_\epsilon(x) &= \frac{\ln(x+\epsilon)-\ln \epsilon}{\ln(1+\epsilon)-\ln \epsilon}.
\end{align*}
$$

The first type of functions have quite large derivatives near $x=0.5$ and will therefore increase the contrast in images with a concentration of intensities near 0.5. 
The second type of functions have a large derivative near $x=0$ and will therefore increase the contrast in images with a large proportion of small intensity values, i.e., very dark images. 
The following code plots the functions $f_4$, $f_{10}$, and $f_{100}$.

In [9]:
def contrast_adjust0(X,n):
    # Assumes that the values are in [0,255]
    X /= 255.
    X -= 1/2.
    X *= n
    arctan(X, X)
    X /= (2*arctan(n/2.)) 
    X += 1/2.0
    X *= 255 # Maps the values back to [0,255]

In [10]:
x=linspace(0, 1, 100)

n=4
f4 = arctan(n*(x-1/2.))/(2*arctan(n/2.)) + 1/2.
n=10
f10=arctan(n*(x-1/2.))/(2*arctan(n/2.)) + 1/2.
n=100
f100=arctan(n*(x-1/2.))/(2*arctan(n/2.)) + 1/2.
plt.plot(x,f4,'r',x,f10,'g',x,f100,'b');
plt.axis([0, 1, 0, 1])
plt.legend(['n=4','n=10','n=100'], loc='lower right', frameon=False)

The following code plots the functions $g_{0.1}$, $g_{0.01}$, and $g_{0.001}$.

In [11]:
eps=0.1
g01=(log(x+eps)-log(eps))/(log(1+eps)-log(eps))
eps=0.01
g001=(log(x+eps)-log(eps))/(log(1+eps)-log(eps))
eps=0.001
g0001=(log(x+eps)-log(eps))/(log(1+eps)-log(eps))
plt.plot(x,g01,'r',x,g001,'g',x,g0001,'b')
plt.axis([0, 1, 0, 1])
plt.legend(['$\epsilon$=0.1','$\epsilon$=0.01','$\epsilon$=0.001'], loc='lower right', frameon=False)

The following code applies $f_{10}$ to the image in the right part of Figure 8.4.

In [12]:
X1 = img.copy()
contrast_adjust0(X1, 10)
imshow(uint8(X1))

The following code applies $g_{0.01}$ to the image in the right part of Figure 8.4.

In [13]:
X2 = img.copy()
contrast_adjust(X2, 0.01)
imshow(uint8(X2))

Since the image was quite well balanced, $f_{10}$ made the dark areas too dark and the bright areas too bright.
$g_{0.01}$ on the other hand has made the image as a whole too bright.

Increasing the contrast is easy to implement. The following function uses the contrast adjusting function $g_\epsilon$, with $\epsilon$ as parameter.

        def contrast_adjust(X,epsilon):
            # Assumes that the values are in [0,255]
            X /= 255.
            X += epsilon
            log(X, X) 
            X -= log(epsilon)
            X /= (log(1+epsilon)-log(epsilon))
            X *= 255


<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 8.6: Generating black and white images
<div id="exercise:genbw"></div>
<!-- keywords = imageschap -->

Black and white images can be generated from grayscale images (with values between $0$ and $255$) by replacing each pixel value with the one of $0$ and $255$ which
is closest. Use this strategy to generate the black and white image shown in 
the right part of Figure 8.2.


<!-- --- begin solution of exercise --- -->
**Solution.**
The following code can be used

In [14]:
X2 = 255*(X1 >= 128)
imshow(uint8(X2))

<!-- --- end solution of exercise --- -->

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 8.7: Adjusting contrast
<div id="exercise:contrastex"></div>
<!-- keywords = imageschap; -->


**a)**
Write a function which instead uses the function $f_n$ from Example 8.5 to adjust the contrast (rather than $g_{\epsilon}$). $n$ should be a parameter to your function.

**b)**
Generate the left and right images in Figure 8.7 on your own by using code which calls the function you have written, as well as `contrast_adjust`.


<!-- --- begin solution of exercise --- -->
**Solution.**
The following code can be used.

In [15]:
X1 = img.copy()
contrast_adjust0(X1, 10)
imshow(uint8(X1))

In [16]:
X2 = img.copy()
contrast_adjust(X2, 0.01)
imshow(uint8(X2))

<!-- --- end solution of exercise --- -->

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 8.8: Adjusting contrast with another function
<div id="exercise:contrastimages3"></div>
<!-- keywords = imageschap; student -->


**a)**
Show that the function $h_n: \mathbb{R} \rightarrow \mathbb{R}$ given by

$$
h_n(x) = x^n,
$$

maps the interval $[0,1] \rightarrow [0,1]$ for any $n$, and that $h_n^\prime (1) \rightarrow \infty$ as $n \rightarrow \infty$.


<!-- --- begin solution of exercise --- -->
**Solution.**
We have that $h_n^\prime(x)=nx^{n-1}$, so that $h_n^\prime(1)=n\to\infty$.

<!-- --- end solution of exercise --- -->

**b)**
The image `secret.jpg`, shown in Figure 8.8, 
contains some information that is nearly invisible to the naked eye on most monitors. Use
the function $h_n$, to reveal the secret message.


<!-- dom:FIGURE: [images/secret.jpg, frac=0.7] Secret message. <div id="imageexercise"></div> -->
<!-- begin figure -->
<div id="imageexercise"></div>

<p>Secret message.</p>
<img src="images/secret.jpg" >

<!-- end figure -->


<!-- --- begin hint in exercise --- -->

**Hint.**
Convert to a grayscale image before you adjust the contrast with $h_n$.

<!-- --- end hint in exercise --- -->


<!-- --- begin solution of exercise --- -->
**Solution.**
The following code can be used

In [17]:
n = 100
X = double(imread('images/secret.jpg', 'jpg'))
X = (X[:,:,0] + X[:,:,1]+ X[:,:,2])/3.
map_to_01(X)
X **= n
X *= 255
X = uint8(X)
imshow(X)

<!-- --- end solution of exercise --- -->

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Example 8.9: Smoothing an image
<div id="example:smoothing"></div>
<!-- keywords = imageschap -->

When we considered filtering of digital sound, low-pass filters dampened high frequencies. 
We will here similarly see that an image can be smoothed by applying a low-pass filters to the rows and the columns.
Let us write down the corresponding computational molecules, and let us use low-pass filters with filter coefficients from Pascal's triangle. 
If we use the filter $S=\frac{1}{4}\{1,\underline{2},1\}$ (row 2 from Pascal's triangle), Theorem 2 gives the computational molecule

$$
A=
\frac{1}{16}
\begin{pmatrix}
1 & 2 & 1\\ 
2 & \underline{4} & 2\\ 
1 & 2 & 1
\end{pmatrix}.
$$

If the pixels in the image are $p_{i,j}$, this means that we compute the new pixels by

$$
\begin{multline*}
\hat p_{i,j}=\frac{1}{16}\bigl(4 p_{i,j}+2(p_{i,j-1}+p_{i-1,j}+p_{i+1,j}+p_{i,j+1})\\ 
+p_{i-1,j-1}+p_{i+1,j-1}+p_{i-1,j+1}+p_{i+1,j+1}\bigr).
\end{multline*}
$$

If we instead use the filter $S=\frac{1}{64}\{1,6,15,\underline{20},15,6,1\}$ (row 6 from Pascal's triangle), we obtain the molecule

$$
\frac{1}{4096}
\begin{pmatrix}
1 & 6 & 15 & 20 & 15 & 6 & 1 \\ 
 6 & 36 & 90 & 120 & 90 & 36 & 6 \\ 
 15 & 90 & 225 & 300 & 225 & 90 & 15 \\ 
 20 & 120 & 300 & \underline{400} & 300 & 120 & 20 \\ 
 15 & 90 & 225 & 300 & 225 & 90 & 15 \\ 
 6 & 36 & 90 & 120 & 90 & 36 & 6 \\ 
 1 & 6 & 15 & 20 & 15 & 6 & 1
\end{pmatrix}.
$$

We anticipate that both molecules give a smoothing effect, but that the second molecule provides more smoothing. 
To make the smoothing effect visible, we have zoomed in on the face in the image. The following function was used to obtain this portion of the image, and will be used repeatedly in the following:

In [18]:
def create_excerpt():
    img = double(imread('images/lena.png','png'))
    return img[128:384,128:384,:]

The result of applying the two molecules above to our grayscale image

In [19]:
excerpt=create_excerpt()
X1 = excerpt.copy()
imshow(uint8(X1))

is shown below

In [20]:
def shortmolecule(x, bd_mode):
    filter_impl(array([1, 2, 1])/4., x, bd_mode)
    
X2 = excerpt.copy()
tensor2_impl(X2, shortmolecule, shortmolecule, 'symm')
imshow(uint8(X2))

In [21]:
X3 = excerpt.copy()

S2 = convolve((1/8.)*array([1, 3, 3, 1]),(1/8.)*array([1, 3, 3, 1]))

def longmolecule(x, bd_mode):
    filter_impl(S2, x, bd_mode)
    
tensor2_impl(X3, longmolecule, longmolecule, 'symm')
imshow(uint8(X3))

The smoothing effect is best visible in the second image. Smoothing effects are perhaps more visible if we use the simple chess pattern image
generated as follows

In [22]:
N=16
img=255*ones((N, N))
img[0:(int(N/2)),0:(int(N/2))] = 0
img[(int(N/2)):N,(int(N/2)):N] = 0
    
X1 = img.copy()
imshow(uint8(X1))

The image can be smoothed out horizontally as follows,

In [23]:
def donothing(x, bd_mode):
    a=1
    
X2 = img.copy()
tensor2_impl(X2, donothing, shortmolecule, 'per')
imshow(uint8(X2))

vertically as follows,

In [24]:
X3 = img.copy()
tensor2_impl(X3, shortmolecule, donothing, 'per')
imshow(uint8(X3))

and both horizontally and vertically as follows.

In [25]:
X4 = img.copy()
tensor2_impl(X4, shortmolecule, shortmolecule, 'per')
imshow(uint8(X4))

Again we have used the filter $S=\frac{1}{4}\{1,\underline{2},1\}$. 
Here we also have shown what happens if we only smooth the image in one of the directions.
In the right image we have smoothed in both directions. 
We then see the union of the two one-dimensional smoothing operations. 










<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Example 8.10: Edge detection
<div id="example:edge_detection"></div>
<!-- keywords = imageschap -->

Another operation on images which can be expressed in terms of computational molecules is *edge detection*. 
An edge in an image is characterized by a large change in intensity values over a small distance in the image. 
For a continuous function this corresponds to a large derivative. 
We cannot compute exact derivatives for an image since it is only defined at isolated points, however,
but we have a perfect situation for numerical differentiation techniques.


### **Partial derivative in $x$-direction**

Let us first compute the partial derivative $\partial P/\partial x$ at all points in the image.
Note first that it is the second coordinate in an image which refers to the $x$-direction you are used to from plotting functions. This means that the much used symmetric Newton quotient
approximation to the partial derivative [[knut]](#knut) takes the form

$$
\frac{\partial P}{\partial x}(i,j)\approx \frac{p_{i,j+1}-p_{i,j-1}}{2},
$$

where we have used the convention $h=1$ (i.e. the derivative measures 'intensity per pixel'). 
This corresponds to applying the high-pass filter $S=\frac{1}{2}\{1,\underline{0},-1\}$ to all the rows, 
alternatively applying the tensor product $I\otimes S$ to the image, i.e. the molecule

$$
\frac{1}{2}
\begin{pmatrix}
0 & 0 & 0\\ 
1 & \underline{0} & -1\\ 
0 & 0 & 0
\end{pmatrix}.
$$

We have included two rows of zeroes to make it clear how the computational molecule is to be applied when we place it over the pixels. 
If we apply this molecule to the usual excerpt of the Lena image

In [26]:
def diffxmolecule(x, bd_mode):
    filter_impl(array([1, 0, -1])/2., x, bd_mode)
    
excerpt=create_excerpt()
X1 = excerpt.copy()
tensor2_impl(X1, donothing, diffxmolecule, 'per')
imshow(uint8(X1))

This image shows many artifacts since the pixel values lie outside the legal range: many of the intensities are in fact negative. Let us therefore normalize and map all intensities to $[0,1]$.

In [27]:
X2 = excerpt.copy()
tensor2_impl(X2, donothing, diffxmolecule, 'per')
map_to_01(X2)
X2 *= 255
imshow(uint8(X2))

The predominant color of this image is an average grey, i.e. an intensity of about 0.5. 
To get more detail in the image we therefore try to increase the contrast by applying the function $f_{50}$ from Example 8.5 to each intensity value. 
This does indeed show more detail.

In [28]:
X3 = excerpt.copy()
tensor2_impl(X3, donothing, diffxmolecule, 'per')
map_to_01(X3)
X3 *= 255
contrast_adjust0(X3,50)
imshow(uint8(X3))

It is important to understand the colors in these images. We have computed the derivative in the $x$-direction, and for this image it turns out that the smallest and largest value of the derivative have about the same absolute value, but with opposite signs. 
A value of 0 in the left image in Figure 8.11 corresponds to no change in intensity between the two pixels, but these will be mapped to about $0.5$. 
In the second image the edges (where the largest values of the derivative appear) have been mapped to black and white, while points with small or no changes in intensity have been mapped to a middle grey-tone. 
The middle image thus tells us that large parts of the image have little variation in intensity. 


### **Partial derivative in $y$-direction**

The partial derivative $\partial P/\partial y$ can be computed analogously to $\partial P/\partial x$,
i.e. we apply the filter $-S=\frac{1}{2}\{-1,\underline{0},1\}$ to all columns of the image 
(alternatively, apply the tensor product $-S\otimes I$ to the image), where $S$ is the filter we
used in the $x$-direction. The positive direction of this axis in an image is opposite to the direction of the $y$-axis we use when plotting functions, 
and this explains the additional minus sign.
We now obtain the molecule

$$
\frac{1}{2}
\begin{pmatrix}
0 & 1 & 0\\ 
0 & \underline{0} & 0\\ 
0 & -1 & 0
\end{pmatrix}.
$$

Let us compare the partial derivatives in both directions.
First we reproduce the partial derivative in $x$-direction.

In [29]:
excerpt=create_excerpt()   
X1 = excerpt.copy()
tensor2_impl(X1, donothing, diffxmolecule, 'per')
map_to_01(X1)
X1 *= 255
contrast_adjust0(X1, 50)
imshow(uint8(X1))

Then we compute the partial derivative in $y$-direction.

In [30]:
def diffymolecule(x, bd_mode):
    filter_impl(array([-1, 0, 1])/2., x, bd_mode)
    
X2 = excerpt.copy()
tensor2_impl(X2, diffymolecule, donothing, 'per')
map_to_01(X2)
X2 *= 255
contrast_adjust0(X2, 50)
imshow(uint8(X2))

The intensities have been normalized and the contrast enhanced by the function $f_{50}$ from Example 8.5.

### **The gradient**

The gradient of a scalar function in two variables is defined by the vector

$$
\nabla P=\Biggl(\frac{\partial P}{\partial x}, \frac{\partial P}{\partial y}\Biggr).
$$

The length of the gradient is

$$
\vert \nabla P\vert = \sqrt{\Biggl(\frac{\partial P}{\partial x}\Biggr)^2 + \Biggl(\frac{\partial P}{\partial y}\Biggr)^2}.
$$

When the two first derivatives have been computed it is thus simple to compute the length of the gradient.  
As for the first order derivatives, it is possible for the length of the gradient to be outside the legal range of values. 
Let us first compute and show the values of the gradient.

In [31]:
excerpt=create_excerpt()
X1 = excerpt.copy()
tensor2_impl(X1, donothing, diffxmolecule, 'per')
X2 = excerpt.copy()
tensor2_impl(X2, diffymolecule, donothing, 'per')
X1 = X1**2 + X2**2
imshow(uint8(X1))

Then we map the intensities to the legal range.

In [32]:
X2[:] = X1
map_to_01(X2)
X2 *= 255
imshow(uint8(X2))

Then we use the function $g_{0.01}$ to adjust the contrast.

In [33]:
X3[:] = X2
contrast_adjust0(X3, 50)
imshow(uint8(X3))

The image of the gradient looks quite different from the images of the two partial derivatives. The reason is that the numbers that represent the length of 
the gradient are (square roots of) sums of squares of numbers.  This means that the parts of the image that have virtually constant intensity 
(partial derivatives close to 0) are colored black. In the images of the partial derivatives these values ended up in the middle of the range of intensity values, with a final color of grey, since there were both positive and negative values.
To enhance the contrast for this image we should thus do something different, like applying a one of the functions in the right plot of Figure 8.6. 

The gradient contains information about both derivatives and therefore emphasizes edges in all directions. It also gives a simpler image since the sign of 
the derivatives has been removed.




<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Example 8.11: Second-order derivatives
<div id="example:secondorder_derivative"></div>
<!-- keywords = imageschap -->

To compute the three second order derivatives we can combine the two computational molecules which we already have described. For the mixed second order derivative we get
$(I\otimes S)((-S)\otimes I)=-S\otimes S$. For the last two second order derivative $\frac{\partial^2P}{\partial x^2}$, $\frac{\partial^2P}{\partial y^2}$, we can also use the
three point approximation to the second derivative [[knut]](#knut)

$$
\frac{\partial P}{\partial x^2}(i,j)\approx p_{i,j+1}-2p_{i,j}+p_{i,j-1}
$$

(again we have set $h=1$). This gives a smaller molecule than if we combine the two molecules for order one differentiation 
(i.e. $(I\otimes S)(I\otimes S)=(I\otimes S^2)$ and $((-S)\otimes I)((-S)\otimes I)=(S^2\otimes I)$), since 
$S^2=\frac{1}{2}\{1,\underline{0},-1\}\frac{1}{2}\{1,\underline{0},-1\}=\frac{1}{4}\{1,0,\underline{-2},0,1\}$. 
The second order derivatives of an image $P$ can thus be computed by applying the computational molecules

$$
\begin{align*}
\frac{\partial^2P}{\partial x^2}:&\qquad
\begin{pmatrix}
0 & 0 & 0\\ 
1 & \underline{-2} & 1\\ 
0 & 0 & 0
\end{pmatrix},  
\\ 
\frac{\partial^2P}{\partial y \partial x}:&\qquad
\frac{1}{4}
\begin{pmatrix}
-1 & 0 & 1\\ 
0 & \underline{0} & 0\\ 
1 & 0 & -1
\end{pmatrix}, 
\\ 
\frac{\partial^2P}{\partial y^2}:&\qquad
\begin{pmatrix}
0 & 1 & 0\\ 
0 & \underline{-2} & 0\\ 
0 & 1 & 0
\end{pmatrix}. 
\end{align*}
$$

With these molecules it is quite easy to compute the second-order derivatives.
First the $xx$-derivative.

In [34]:
excerpt=create_excerpt() 
X1 = excerpt.copy()
tensor2_impl(X1, donothing, diffxmolecule, 'per')
tensor2_impl(X1, donothing, diffxmolecule, 'per')
map_to_01(X1)
X1 *= 255
contrast_adjust0(X1, 100)
imshow(uint8(X1))

Then the $xy$-derivative.

In [35]:
X2 = excerpt.copy()
tensor2_impl(X2, diffymolecule, diffxmolecule, 'per')
map_to_01(X2)
X2 *= 255
contrast_adjust0(X2, 100)
imshow(uint8(X2))

Then the $yy$-derivative.

In [36]:
X3 = excerpt.copy()
tensor2_impl(X3, diffymolecule, donothing, 'per')
tensor2_impl(X3, diffymolecule, donothing, 'per')
map_to_01(X3)
X3 *= 255
contrast_adjust0(X3, 100)
imshow(uint8(X3))

The computed derivatives were first normalized and then the contrast enhanced with the function $f_{100}$ from Example 8.5.

As for the first derivatives, the $xx$-derivative seems to emphasize vertical edges and the $yy$-derivative horizontal edges. However, we also see that the second derivatives are
more sensitive to noise in the image (the areas of grey are less uniform). The mixed derivative behaves a bit differently from the other two, and not surprisingly it seems to pick
up both horizontal and vertical edges.

This procedure can be generalized to higher order derivatives also. To apply $\frac{\partial^{k+l}P}{\partial x^k\partial y^l}$ to an image we can compute $S_l\otimes S_k$ where
$S_r$ corresponds to any point method for computing the $r$'th order derivative. We can also compute $(S^l)\otimes(S^k)$, where we iterate the
filter $S=\frac{1}{2}\{1,\underline{0},-1\}$ for the first derivative. As pointed out, this gives longer filters.

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Example 8.12: A chess pattern image
<div id="example:chesspatternmoluecules"></div>
<!-- keywords = imageschap -->

Let us also apply the molecules for differentiation to the chess pattern test image produced by the code

In [37]:
N = 128;
img=zeros((N, N))
for x in range(N):
    for y in range(N):
        img[x,y] = 255*( (mod(x-1,64)>=32) == (mod(y-1,64)>=32) )

and looks as follows

In [38]:
X11 = img.copy()
imshow(uint8(X11))

We first apply $S\otimes I$.

In [39]:
X12 = img.copy()
tensor2_impl(X12, diffymolecule, donothing, 'per')
map_to_01(X12)
X12 *= 255
imshow(uint8(X12))

Then we apply $I\otimes S$.

In [40]:
X13 = img.copy()
tensor2_impl(X13, donothing, diffxmolecule, 'per')
map_to_01(X13)
X13 *= 255
imshow(uint8(X13))

Then we apply $S\otimes S$.

In [41]:
X21 = img.copy()
tensor2_impl(X21, diffymolecule, diffxmolecule, 'per')
map_to_01(X21)
X21 *= 255
imshow(uint8(X21))

Then we apply $I\otimes S^2$.

In [42]:
X22 = img.copy()
tensor2_impl(X22, donothing, diffxmolecule, 'per')
tensor2_impl(X22, donothing, diffxmolecule, 'per')
map_to_01(X22)
X22 *= 255
imshow(uint8(X22))

The we apply $S^2\otimes I$.

In [43]:
X23 = img.copy()
tensor2_impl(X23, diffymolecule, donothing, 'per')
tensor2_impl(X23, diffymolecule, donothing, 'per')
map_to_01(X23)
X23 *= 255
imshow(uint8(X23))

These images make it even clearer that 
* $S\otimes I$ detects horizontal edges, 

* $I\otimes S$ detects vertical edges, 

* $S\otimes S$ detects all points where abrupt changes appear in both directions. 

We also see that the second order partial derivative detects exactly the same edges as the first order
partial derivative, and that the edges detected with $I\otimes S^2$ are wider than the ones detected with $I\otimes S$. The reason is that the filter $S^2$
has more filter coefficients than $S$. 

Edges are detected with different colors, reflecting whether the difference between the neighboring pixels is positive or negative. 
Note also the additional edges at the first and last rows/edges in the images. The reason is that the filter $S$ is defined by assuming that the pixels repeat periodically (i.e. it is a
circulant Toeplitz matrix). 

Defining a two-dimensional filter by filtering columns and then rows is not the only way we can define a two-dimensional filter. Another possible way is to let the $MN\times
MN$-matrix itself be a filter. Unfortunately, this is a bad way to define filtering, since there are some undesirable effects near the boundaries between rows:
in the vector we form, the last element of one row is followed by the first element of the next row. These boundary effects are unfortunate when a filter is applied.

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 8.13: Generating images
<div id="exercise:genimgs"></div>
<!-- keywords = imageschap; student -->

Write code which calls the function `tensor2_impl` with appropriate filters to produce the following images:


**a)**
The right image in Figure 8.9.


<!-- --- begin solution of exercise --- -->
**Solution.**
The following code can be used

In [44]:
X3 = excerpt.copy()

S2 = convolve((1/8.)*array([1, 3, 3, 1]),(1/8.)*array([1, 3, 3, 1]))

def longmolecule(x, bd_mode):
    filter_impl(S2, x, bd_mode)
    
tensor2_impl(X3, longmolecule, longmolecule, 'symm')
imshow(uint8(X3))

<!-- --- end solution of exercise --- -->

**b)**
The right image in Figure 8.11.


<!-- --- begin solution of exercise --- -->
**Solution.**
The following code can be used

In [45]:
X3 = excerpt.copy()
tensor2_impl(X3, donothing, diffxmolecule, 'per')
map_to_01(X3)
X3 *= 255
contrast_adjust0(X3,50)
imshow(uint8(X3))

<!-- --- end solution of exercise --- -->

**c)**
The images in Figure 8.13.


<!-- --- begin solution of exercise --- -->
**Solution.**
The following code can be used

In [46]:
excerpt=create_excerpt()
X1 = excerpt.copy()
tensor2_impl(X1, donothing, diffxmolecule, 'per')
X2 = excerpt.copy()
tensor2_impl(X2, diffymolecule, donothing, 'per')
X1 = X1**2 + X2**2
imshow(uint8(X1))

In [47]:
X2[:] = X1
map_to_01(X2)
X2 *= 255
imshow(uint8(X2))

In [48]:
X3[:] = X2
contrast_adjust0(X3, 50)
imshow(uint8(X3))

<!-- --- end solution of exercise --- -->

**d)**
The images in Figure 8.12.


<!-- --- begin solution of exercise --- -->
**Solution.**
The following code can be used

In [49]:
excerpt=create_excerpt()   
X1 = excerpt.copy()
tensor2_impl(X1, donothing, diffxmolecule, 'per')
map_to_01(X1)
X1 *= 255
contrast_adjust0(X1, 50)
imshow(uint8(X1))

In [50]:
def diffymolecule(x, bd_mode):
    filter_impl(array([-1, 0, 1])/2., x, bd_mode)
    
X2 = excerpt.copy()
tensor2_impl(X2, diffymolecule, donothing, 'per')
map_to_01(X2)
X2 *= 255
contrast_adjust0(X2, 50)
imshow(uint8(X2))

<!-- --- end solution of exercise --- -->

**e)**
The images in Figure 8.14.


<!-- --- begin solution of exercise --- -->
**Solution.**
The following code can be used

In [51]:
excerpt=create_excerpt() 
X1 = excerpt.copy()
tensor2_impl(X1, donothing, diffxmolecule, 'per')
tensor2_impl(X1, donothing, diffxmolecule, 'per')
map_to_01(X1)
X1 *= 255
contrast_adjust0(X1, 100)
imshow(uint8(X1))

In [52]:
X2 = excerpt.copy()
tensor2_impl(X2, diffymolecule, diffxmolecule, 'per')
map_to_01(X2)
X2 *= 255
contrast_adjust0(X2, 100)
imshow(uint8(X2))

In [53]:
X3 = excerpt.copy()
tensor2_impl(X3, diffymolecule, donothing, 'per')
tensor2_impl(X3, diffymolecule, donothing, 'per')
map_to_01(X3)
X3 *= 255
contrast_adjust0(X3, 100)
imshow(uint8(X3))

<!-- --- end solution of exercise --- -->

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Example 8.22: Change of coordinates with the DFT
<div id="example:change_coords_dft2"></div>
<!-- keywords = imageschap -->

The DFT was defined as a change of coordinates from the standard basis to the Fourier basis. Let us substitute the DFT and the IDFT for $S_1$, $S_2$.

Modern image standards do typically not apply a change of coordinates to the entire image. 
Rather the image is split into smaller squares of appropriate size, called blocks, and a
change of coordinates is performed independently on each block. In this example we will split the image into blocks of size $8\times 8$. 

Recall that the DFT values express frequency components. The same applies for the 2D DFT and thus for images, but 
frequencies are now represented in two different directions. Let us make a parallel to Example 2.5 for the Lena image, i.e. 
we will view the image after a 2D DFT, followed by discarding DFT coefficients below a given threshold, followed by a 2D IDFT.
As for sound this should have little effect
on the human perception of the image, if the threshold is chosen with care. 
DFT-coefficients in a matrix `X` below a threshold can be discarded with the following code:

        X *= (abs(X) >= threshold)


\verb+abs(X)>=threshold+ returns a *threshold matrix* with $1$ and $0$ of the same size as `X`. 

The following code generates images using threshold values of 1200, 200, and 400, respectively.

In [54]:
def dft_impl8(x, bd_mode):
    N = shape(x)[0]
    for n in range(0, N, 8):
        x[n:(n+8), :] = fft.fft(x[n:(n+8), :], axis=0)

In [55]:
def idft_impl8(x, bd_mode):
    N = shape(x)[0]
    for n in range(0, N, 8):
        x[n:(n+8), :] = fft.ifft(x[n:(n+8), :], axis=0)

In [56]:
X = create_excerpt()
X = X.astype(complex)
X1 = X.copy()
forw_comp_rev_2d(X1, dft_impl8, idft_impl8, 100)
imshow(uint8(abs(X1)))

In [57]:
X2 = X.copy()
forw_comp_rev_2d(X2, dft_impl8, idft_impl8, 200)
imshow(uint8(abs(X2)))

In [58]:
X3 = X.copy()
forw_comp_rev_2d(X3, dft_impl8, idft_impl8, 400)
imshow(uint8(X3))

When increasing the threshold, the image becomes more and more unclear, but the image is quite clear in 
the first case, where as much as more than $76.6\%$ of the samples have been zeroed out. A blocking effect at the block boundaries is clearly visible.

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Example 8.23: Change of coordinates with the DCT
<div id="example:change_coords_dct2"></div>
<!-- keywords = imageschap -->

Similarly to the DFT, the DCT was the change of coordinates from the standard basis to what we called the DCT basis. Let us substitute the DCT and the IDCT for $S_1$, $S_2$.

The DCT is used more than the DFT in image processing. In particular, the JPEG standard applies a two-dimensional DCT, rather than a two-dimensional DFT. 
With the JPEG standard, the blocks are always $8\times 8$, as in the previous example. It is of course not a coincidence that a power of $2$ is chosen here: the DCT, as the
DFT, has an efficient implementation for powers of $2$.

If we follow the same strategy for the DCT as above, i.e. zero out DCT-coefficients below a given threshold[^jpegcomment], 
the following code generates images using threshold values of 30, 50, and 100, respectively.

In [59]:
def dct_impl8(x, bd_mode):
    N = shape(x)[0]
    for n in range(0, N, 8):
        x[n:(n+8), :] = dct(x[n:(n+8), :], norm='ortho', axis=0)

In [60]:
def idct_impl8(x, bd_mode):
    N = shape(x)[0]
    for n in range(0, N, 8):
        x[n:(n+8), :] = idct(x[n:(n+8), :], norm='ortho', axis=0)

In [61]:
X = create_excerpt()
X1 = X.copy()
forw_comp_rev_2d(X1, dct_impl8, idct_impl8, 30)
imshow(uint8(X1))

In [62]:
X2 = X.copy()
forw_comp_rev_2d(X2, dct_impl8, idct_impl8, 50)
imshow(uint8(X2))

In [63]:
X3 = X.copy()
forw_comp_rev_2d(X3, dct_impl8, idct_impl8, 100)
imshow(uint8(X3))

It is also interesting to see what happens if we don't split the image into blocks. Of course, when we discard many of the DCT-coefficients, we
should see some artifacts, but there is no reason to believe that these occur at the old block boundaries. 
The new artifacts can be seen 
with the following code, and take a completely different shape.

In [64]:
def dct_impl_full(x, bd_mode):
    x[:] = dct(x, norm='ortho', axis=0)

In [65]:
def idct_impl_full(x, bd_mode):
    x[:] = idct(x, norm='ortho', axis=0)

In [66]:
X1 = X.copy()
forw_comp_rev_2d(X1, dct_impl_full, idct_impl_full, 30)
imshow(uint8(X1))

In [67]:
X2 = X.copy()
forw_comp_rev_2d(X2, dct_impl_full, idct_impl_full, 50)
imshow(uint8(X2))

In [68]:
X3 = X.copy()
forw_comp_rev_2d(X3, dct_impl_full, idct_impl_full, 100)
imshow(uint8(X3))

In the exercises you will be asked to write code which generates the images from these examples.

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 8.24: Implementing DFT and DCT on blocks
<div id="exercise:dftdctonblocks"></div>
<!-- keywords = imageschap; student -->

In this section we have used functions which apply the DCT and the DFT either to subblocks of size $8\times 8$, or to the full image.


**a)**
Implement functions `dft_impl8`, `idft_impl8`, `dct_impl8`, and `idct_impl8` 
which apply the DFT, IDFT, DCT, and IDCT, to consecutive blocks of length $8$.









**b)**
Implement the two-dimensional FFT, IFFT, DCT, and IDCT on images, with the help of their one-dimensional counterparts, as well as the function `tensor2_impl`.


<!-- --- begin solution of exercise --- -->
**Solution.**
The following code can be used.

In [69]:
def dft_impl_full(x, bd_mode):
    x[:] = fft.fft(x, axis=0)

In [70]:
def idft_impl_full(x, bd_mode):
    x[:] = fft.ifft(x, axis=0)

In [71]:
tensor2_impl(X, dft_impl_full, dft_impl_full, 'symm')
tensor2_impl(X, idft_impl_full, idft_impl_full, 'symm')

In [72]:
tensor2_impl(X, dct_impl_full, dct_impl_full, 'symm')
tensor2_impl(X, idct_impl_full, idct_impl_full, 'symm')

<!-- --- end solution of exercise --- -->

**c)**
The function `forw_comp_rev_2d` in the library applies given transforms to the rows and columns of an input image, 
sets the coefficients below a certain threshold to zero, and transforms back.
Run `forw_comp_rev_2d` for different threshold parameters on the sample image, 
and with the functions `dct_impl8`, `idct_impl8`, as well as DCT and IDCT applied to the entire input, as parameters. 
Check that this reproduces the DCT test images of this section, 
and that the correct numbers of values which have been discarded (i.e. which are below the threshold) are printed on screen.


<!-- --- begin solution of exercise --- -->
**Solution.**
The following code can be used.

In [73]:
X = create_excerpt()
X1 = X.copy()
forw_comp_rev_2d(X1, dct_impl8, idct_impl8, 30)
imshow(uint8(X1))

In [74]:
X2 = X.copy()
forw_comp_rev_2d(X2, dct_impl8, idct_impl8, 50)
imshow(uint8(X2))

In [75]:
X3 = X.copy()
forw_comp_rev_2d(X3, dct_impl8, idct_impl8, 100)
imshow(uint8(X3))

In [76]:
X1 = X.copy()
forw_comp_rev_2d(X1, dct_impl_full, idct_impl_full, 30)
imshow(uint8(X1))

In [77]:
X2 = X.copy()
forw_comp_rev_2d(X2, dct_impl_full, idct_impl_full, 50)
imshow(uint8(X2))

In [78]:
X3 = X.copy()
forw_comp_rev_2d(X3, dct_impl_full, idct_impl_full, 100)
imshow(uint8(X3))

<!-- --- end solution of exercise --- -->


<!-- --- end exercise --- -->